In [3]:
import rasterio
from rasterio.windows import from_bounds

In [4]:
## Eaton
# # input_sar_image_path  = "../SAR_Data_processing_eaton/8_subset/eaton_S1A_IW_GRDH_1SDV_stack_32611_clip_bilinear_10m.tif"
# input_sar_image_path  = "input/eaton_s1_ready.tif"
# glcm_raster_path="../SAR_Data_processing_eaton/9_glcm/eaton_S1A_IW_GRDH_1SDV_Cal_Spk_TC_stack_glcm_32611_bilinear_10.tif"
# thermal_image_path="../ThermalData/03_diff/dTRAD.tif"
# dnbr_image_path="../S2_Data_Processing/Indices/04_CLIP/dNBR_bilinear_10_clip.tif"
# prefix="eaton"
# Define , eaton
# minx, miny, maxx, maxy=390284.9643946937, 3778493.0553008374, 407654.9643946937, 3791083.0553008374


## Palisades
input_sar_image_path  = "../../SAR_Data_processing/8_1_project/S1A_IW_GRDH_1SDV_palisades_32611_bilinear_10.tif"
glcm_raster_path=""
thermal_image_path="../../ThermalData/03_diff/dTRAD.tif"
dnbr_image_path="../../S2_Data_Processing/Indices/04_CLIP/dNBR_bilinear_10_clip.tif"
prefix="palisades"

## palisades
minx, miny, maxx, maxy=330142.6891275004,3757093.028339643,375622.6891275004,3794643.028339643


In [5]:

images_dict={f"{prefix}_s1_ready":input_sar_image_path,
            #  "glcm_raster_path":glcm_raster_path,
             f"{prefix}_thermal_ready":thermal_image_path,
            f"{prefix}_dnbr_ready":dnbr_image_path,
             }

# images_dict={"input_sar_image_path":input_sar_image_path,
#              "glcm_raster_path":"../SAR-Burnt-Area-Mapping/input/eaton_glcm_ready.tif",
#              "thermal_image_path":"../SAR-Burnt-Area-Mapping/input/eaton_thermal_ready.tif",
#              "dnbr_image_path":"../SAR-Burnt-Area-Mapping/input/eaton_dnbr_ready.tif",
#              }


### save in order: VH_PRE, VV_PRE, VH_POST, VV_POST

In [ ]:

# # save in order: VH_PRE, VV_PRE, VH_POST, VV_POST
# output_ordered_image_path="input/eaton_s1_ready.tif"
# with rasterio.open(input_sar_image_path) as dataset:
#     vh_post_band=dataset.read(1)
#     vv_post_band=dataset.read(2)
#     vh_pre_band=dataset.read(3)
#     vv_pre_band=dataset.read(4)
#     meta=dataset.meta.copy()

# # Write the clipped data to a new file
# with rasterio.open(output_ordered_image_path, 'w', **meta) as dest:
#     dest.write(vh_pre_band, 1)  
#     dest.write(vv_pre_band, 2)  
#     dest.write(vh_post_band, 3)  
#     dest.write(vv_post_band, 4)  


In [6]:
for image_key in list(images_dict.keys()):
    print(image_key)

    with rasterio.open(images_dict[image_key]) as dataset:
        # Print the properties
        print(f"CRS: {dataset.crs}")
        print(f"Width: {dataset.width}")
        print(f"Height: {dataset.height}")
        print(f"Extent: {dataset.bounds}")
        print(f"Resolution: { dataset.res}")
        print("----------------------------------------")

palisades_s1_ready
CRS: EPSG:32611
Width: 4548
Height: 3755
Extent: BoundingBox(left=330142.6891275004, bottom=3757093.028339643, right=375622.6891275004, top=3794643.028339643)
Resolution: (10.0, 10.0)
----------------------------------------
palisades_thermal_ready
CRS: EPSG:32611
Width: 8833
Height: 5044
Extent: BoundingBox(left=326605.0, bottom=3755115.0, right=414935.0, top=3805555.0)
Resolution: (10.0, 10.0)
----------------------------------------
palisades_dnbr_ready
CRS: EPSG:32611
Width: 7917
Height: 4406
Extent: BoundingBox(left=329840.0, bottom=3753440.0, right=409010.0, top=3797500.0)
Resolution: (10.0, 10.0)
----------------------------------------


## Clipping data by extent

In [5]:
# # Define , eaton
# # minx, miny, maxx, maxy=390284.9643946937, 3778493.0553008374, 407654.9643946937, 3791083.0553008374

# ## palisades
# minx, miny, maxx, maxy=330142.6891275004,3757093.028339643,375622.6891275004,3794643.028339643

clip_extent = (minx, miny, maxx, maxy)

In [ ]:
# first_key = next(iter(images_dict))

# # Remove it
# images_dict.pop(first_key)
# print(images_dict)

In [6]:
# images_dict_1={
#              "eaton_glcm_ready":glcm_raster_path,
#              "eaton_thermal_ready":thermal_image_path,
#              "eaton_dnbr_ready":dnbr_image_path,
#              }

for image_key in list(images_dict.keys()):
    # Open the raster dataset
    with rasterio.open(images_dict[image_key]) as dataset:
        # Get the window for the specified extent
        window = from_bounds(*clip_extent, dataset.transform)
        # Read the data in that window
        # clipped_data = dataset.read(1, window=window)  # Read the first band in the window
        
        # Get the new transform for the clipped data
        clipped_transform = dataset.window_transform(window)
        clipped_data = dataset.read(1, window=window)
        # Define the metadata for the clipped raster
        clipped_meta = dataset.meta.copy()
        clipped_meta.update({
            'height': clipped_data.shape[0],
            'width': clipped_data.shape[1],
            'transform': clipped_transform
        })
        count=dataset.count

        # bands = dataset.read()
        
        # Write the clipped data to a new file
        with rasterio.open(f'input/{image_key}.tif', 'w', **clipped_meta) as dest:
            for i in range(count):
                clipped_data = dataset.read(i+1, window=window)
                dest.write(clipped_data, i+1)  # Write the first band
